In [1]:
import xml.etree.ElementTree as ET
from PIL import Image
import os
import pandas as pd
import numpy as np
import json
from tqdm.notebook import tqdm
import cv2

In [2]:
anno_pathes_json = []
img_pathes = {}
file_name = '.json'
img_name = '.jpg'

absolute_path = 'D:\\Datasets\\NCAA\\NCAA_1_jersey\\second_task'
crop_path = 'D:\\Datasets\\crops'
img_folder_path = 'D:\\Datasets\\NCAA\\NCAA_1_tracking\\playerTrackingFrames\\2019.01.05-Clemson_at_Duke'

if not os.path.exists(crop_path):
    os.makedirs(crop_path)

for root, dirs, files in os.walk(absolute_path):      
    for file in files:
        # проверяем, есть ли заданный файл в списке файлов текущей папки
        if (file_name in file) & ('.jsonl' not in file):
            # если есть, то добавляем путь в список
            anno_pathes_json.append(os.path.join(root, file)) 

for root, dirs, files in os.walk(img_folder_path):        
    for file in files:
        # проверяем, есть ли заданный файл в списке файлов текущей папки
        if (img_name in file):
            # если есть, то добавляем путь в список
            sh_file = file.replace('frame_', '')
            img_pathes[sh_file] = os.path.join(root, file)  

In [ ]:
alpha = 0.1
num = 0
annos_list = []
anno_dict = {'file_name': [], 'text': []}
pathes_list = []

for anno in tqdm(anno_pathes_json):
    with open(anno, 'r') as f:
        data = json.load(f)

    #Преобразуем json  в dataFrame
    points = pd.read_json(json.dumps(data['shapes']))
    img_name = str(json.dumps(data['imagePath']))
    img_path = img_name.replace('"', '')
    img_path = os.path.split(img_path)
    img_path = img_pathes[img_path[-1]]

    img = Image.open(img_path)
    

    for j in range(len(points)):
        out_file = {}    

        if points["shape_type"][j] == "rectangle":                       
        
            label = points['label']
            
            split_label = label[j].split('_')
            if len(split_label) != 3:
                continue
            number = split_label.pop()
            
            point_obj = points['points']
            singl_point = point_obj[j]
            [xtl, ytl], [xbr, ybr] = singl_point

            x_dif = np.abs(xtl - xbr)
            y_dif = np.abs(ytl - ybr)

            xtl = int(xtl - alpha * (x_dif / 2))
            xbr = int(xbr + alpha * (x_dif / 2))
            ytl = int(ytl - alpha * (y_dif / 2))
            ybr = int(ybr + alpha * (y_dif / 2))

            # Сделаем кроп номера на футболке
            croppped_image = img.crop((xtl, ytl, xbr, ybr))
            if str(number) == '5(1)':
                number = '5' 
            full_path = crop_path + '\\' + str(number)                

            if not os.path.exists(full_path):
                os.makedirs(full_path)  
            
            res_path = full_path +'\\' + f'ncaa_{str(num)}example.jpg'
            num += 1

            annos_list.append(str(number))
            pathes_list.append(res_path )
            anno_dict['text'] = annos_list
            anno_dict['file_name'] = pathes_list
                
            croppped_image.save(res_path)

df = pd.DataFrame(anno_dict, index=None)
df.to_csv("anno_ncaa.csv")
df.head()  